# Installing requirements

TEST

In [52]:
import os
#TEST

In [55]:
os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

In [56]:
path = os.getcwd()
os.chdir(path+"/code")

In [57]:
os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2/code'

In [58]:
%pip install -r requirements.txt


[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
os.chdir("..")
os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

# Imports

In [60]:
from tabulate import tabulate
import mysql.connector as mysql

# Class DBConnector

In [61]:
class DbConnector:
    """
    Connects to the MySQL server on the Ubuntu virtual machine.
    Connector needs HOST, DATABASE, USER and PASSWORD to connect,
    while PORT is optional and should be 3306.

    Example:
    HOST = "tdt4225-00.idi.ntnu.no" // Your server IP address/domain name
    DATABASE = "testdb" // Database name, if you just want to connect to MySQL server, leave it empty
    USER = "testuser" // This is the user you created and added privileges for
    PASSWORD = "test123" // The password you set for said user
    """

    def __init__(self,
                 HOST="tdt4225-19.idi.ntnu.no",
                 DATABASE="test_19",
                 USER="team19",
                 PASSWORD="team19*"):
        # Connect to the database
        try:
            self.db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD, port=3306)
        except Exception as e:
            print("ERROR: Failed to connect to db:", e)

        # Get the db cursor
        self.cursor = self.db_connection.cursor()

        print("Connected to:", self.db_connection.get_server_info())
        # get database information
        self.cursor.execute("select database();")
        database_name = self.cursor.fetchone()
        print("You are connected to the database:", database_name)
        print("-----------------------------------------------\n")

    def close_connection(self):
        # close the cursor
        self.cursor.close()
        # close the DB connection
        self.db_connection.close()
        print("\n-----------------------------------------------")
        print("Connection to %s is closed" % self.db_connection.get_server_info())


# Example program

In [85]:
class ExampleProgram:

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

    def create_table(self, table_name):
        query = """CREATE TABLE IF NOT EXISTS %s (
                   id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
                   name VARCHAR(30))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % table_name)
        self.db_connection.commit()

    def insert_data(self, table_name):
        names = ['Bobby', 'Mc', 'McSmack', 'Board']
        for name in names:
            # Take note that the name is wrapped in '' --> '%s' because it is a string,
            # while an int would be %s etc
            query = "INSERT INTO %s (name) VALUES ('%s')"
            self.cursor.execute(query % (table_name, name))
        self.db_connection.commit()

    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


def main():
    program = None
    try:
        program = ExampleProgram()
        program.create_table(table_name="Person")
        program.insert_data(table_name="Person")
        _ = program.fetch_data(table_name="Person")
        program.drop_table(table_name="Person")
        # Check that the table is dropped
        program.show_tables()
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()


In [86]:
if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

Data from table Person, raw format:
[(1, 'Bobby'), (2, 'Mc'), (3, 'McSmack'), (4, 'Board')]
Data from table Person, tabulated:
  id  name
----  -------
   1  Bobby
   2  Mc
   3  McSmack
   4  Board
Dropping table Person...
Tables_in_test_19
-------------------
Activity_s
Activity_v
TrackPoint_s
TrackPoint_v
User_s
User_v

-----------------------------------------------
Connection to None is closed


# Task 1

In [89]:
class Task1:

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor


    def create_table_user(self):
        query = """CREATE TABLE IF NOT EXISTS User_v (
                    id INT PRIMARY KEY,
                    has_labels BOOLEAN)
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query)
        self.db_connection.commit()
    

    def create_table_activity(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                   id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
                   name VARCHAR(30))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % "Activity_v")
        self.db_connection.commit()
    

    def create_table_trackpoint(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                   id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
                   name VARCHAR(30))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % "TrackPoint_v")
        self.db_connection.commit()


    def insert_data_user(self):
        # we go to the Data directory
        path = os.getcwd()
        os.chdir(path+"/dataset/dataset/Data")

        # we get the list of all the diferent directory names (users ids) and we sort the list
        id =  os.listdir()
        id.sort()

        # we go back to the dataset directory and we read the labeled_ids.txt file
        os.chdir("..")
        with open('labeled_ids.txt') as f:
            labeled_ids = f.readlines()
        f.close()
        os.chdir("..")
        os.chdir("..")
        
        # we delete the \n in each string
        for i in range (0, len(labeled_ids)):
            labeled_ids[i] = labeled_ids[i].strip()

        # we check if each user has a label or not and we save the info in a list
        # the indexes of has_labels and id lists are correponding
        has_labels = []
        for i in id : 
            if i in labeled_ids : 
                has_labels.append(True)
            else:
                has_labels.append(False)

        # we execute the query to insert the data
        for i in range(len(id)):
            query = "INSERT INTO User_v (id, has_labels) VALUES (%s, %s)"
            values = (id[i], has_labels[i])
            self.cursor.execute(query, values)

        self.db_connection.commit()


    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows


    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)


    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


In [92]:
def main():
    program = None
    try:
        program = Task1()
        program.drop_table("User_v")
        program.create_table_user()
        program.insert_data_user()
        program.fetch_data("User_v")

        
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()

if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

Dropping table User_v...
Data from table User_v, raw format:
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 1), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 1), (21, 1), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (51, 0), (52, 1), (53, 1), (54, 0), (55, 0), (56, 1), (57, 0), (58, 1), (59, 1), (60, 1), (61, 0), (62, 1), (63, 0), (64, 1), (65, 1), (66, 0), (67, 1), (68, 1), (69, 1), (70, 0), (71, 0), (72, 0), (73, 1), (74, 0), (75, 1), (76, 1), (77, 0), (78, 1), (79, 0), (80, 1), (81, 1), (82, 1), (83, 0), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90

In [108]:
os.chdir("..")
os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

In [109]:
path = os.getcwd()
os.chdir(path+"/dataset/dataset/Data")

In [133]:
from datetime import datetime
user_id = []
transportation_mode = []
start_date_time = []
end_date_time = []

for dirpath, dirnames, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if filename.endswith('.txt'):
            with open(os.path.join(dirpath, filename)) as f:
                lines = f.readlines()
                # skip the header
                lines = lines[1:]
                for line in lines:

                    data = line.split()

                    last_directory_name = os.path.basename(dirpath)
                    user_id.append(last_directory_name)

                    transportation_mode.append(data[4])

                    start_date = data[0]
                    start_time = data[1]
                    end_date = data[2]
                    end_time = data[3]
                    start_datetime_str = start_date + " " + start_time
                    end_datetime_str = end_date + " " + end_time
                    combined_start_datetime = datetime.strptime(start_datetime_str, "%Y/%m/%d %H:%M:%S")
                    combined_end_datetime = datetime.strptime(end_datetime_str, "%Y/%m/%d %H:%M:%S")

                    start_date_time.append(combined_start_datetime)
                    end_date_time.append(combined_end_datetime)
            f.close()

In [136]:
len(user_id)

14718

In [137]:
len(transportation_mode)

14718

In [138]:
len(start_date_time)

14718